In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

import torch
from tqdm import tqdm
from routines.routine import Routine
from common.cfg_utils import get_config

cfg = get_config("../configs")
# routine = Routine(cfg)

In [9]:
from datasets.bigbind_act import BigBindActDataset
cfg.data.use_lig = False
dataset = BigBindActDataset(cfg, "val")
dataset[10].lig.ndata.cat_feat.shape

torch.Size([46, 5])